In [1]:
import pandas as pd
import numpy as np
import pytimetk as pytimetk
import random
import matplotlib.pyplot as plt
import os
import datetime
from datetime import date, timedelta 
from datetime import date, timedelta 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error




In [2]:


min_temp_file_path = 'G:/fresh_start/main_thesis/data/raw/Min_temp.xlsx'


intial_min_temp_df = pd.ExcelFile(min_temp_file_path)

In [3]:

min_file_column_order =  [ 'Station NO', 'Min T']
initial_min_dict = {}
# Iterate over each sheet and create a DataFrame
for sheet_name in intial_min_temp_df.sheet_names:
    min_df_parsed = intial_min_temp_df.parse(sheet_name)

    initial_min_dict[sheet_name] = min_df_parsed
    



     # Converting a Year , Month and Day Column into Date column
    min_df_parsed['Date'] = pd.to_datetime(min_df_parsed[['Year', 'Month', 'Day']])

    

    # Set "Date" as the index column
    min_df_parsed.set_index('Date', inplace=True)

    min_df_parsed = min_df_parsed[min_file_column_order]
    
    min_df_parsed= min_df_parsed.rename(columns={'Station NO': 'Station'})
    
    initial_min_dict[sheet_name] = min_df_parsed





In [5]:
min_outlier_check_dict = {}

for station_name, df_min_temp in initial_min_dict.items():
    # Calculate the IQR for 'Min T' column (replace 'Max T' with 'Min T')
    Q1 = df_min_temp['Min T'].quantile(0.25)  # Change 'Max T' to 'Min T'
    Q3 = df_min_temp['Min T'].quantile(0.75)  # Change 'Max T' to 'Min T'
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter out data points outside the bounds
    df_min_temp = df_min_temp[(df_min_temp['Min T'] >= lower_bound) & (df_min_temp['Min T'] <= upper_bound)]  # Change 'Max T' to 'Min T'
    min_outlier_check_dict[station_name] = df_min_temp


In [4]:
def generate_dummy_temp_data( station_code):
    # Create an empty list to store the data
    data = []

    # Generate random temperature data for each day within the date range
    current_date = date(1962, 1, 1)
    while current_date <= date(2022, 12, 31):
        temperature = round(random.uniform(1900, 2000), 1)
        data.append([current_date, station_code, temperature])
        current_date += timedelta(days=1)

    # Create a DataFrame from the generated data
    dummy_temp_df = pd.DataFrame(data, columns=['Date', 'Station', 'Max_Temp'])

    # Set the 'Date' column as the index
    #dummy_temp_df.set_index('Date', inplace=True)

    return dummy_temp_df



In [7]:
min_temp_1960_2022_initial = {}

for station_name, min_temp_df in min_outlier_check_dict.items():
    
    min_temp_df = min_temp_df.reset_index()
    min_temp_df['Date'] = pd.to_datetime(min_temp_df['Date'])
    
    dummy_temp_df = generate_dummy_temp_data(station_name)
    
    dummy_temp_df['Date'] = pd.to_datetime(dummy_temp_df['Date'])
    
    df3 = pd.merge(dummy_temp_df, min_temp_df, on=['Date'], how='left').fillna('NA')
    
    min_temp_1960_2022_initial[station_name] = df3

In [9]:
# Replace 'Max T' with 'Min T'
for station_name, min_temp_df in min_temp_1960_2022_initial.items():
    min_temp_df = min_temp_df[['Date', 'Min T']]
    min_temp_1960_2022_initial[station_name] = min_temp_df

# Initializing data frames
all_station_data_initial_df = []

for station_name, min_temp_df in min_temp_1960_2022_initial.items():
    min_temp_df.loc[:, 'Station'] = station_name
    all_station_data_initial_df.append(min_temp_df)

result_df_min_temp = pd.concat(all_station_data_initial_df, ignore_index=True)

result_df_min_temp = result_df_min_temp[['Date', 'Station', 'Min T']]
# Replace 'NA' with NaN
result_df_min_temp['Min T'] = result_df_min_temp['Min T'].replace('NA', np.nan)
    

In [11]:
result_df_min_temp['Date'] = pd.to_datetime(result_df_min_temp['Date'])

start_date = pd.to_datetime('1990-01-01')
end_date = pd.to_datetime('2022-12-31')

# Filter the DataFrame to include data within the specified date range
filtered_data_df = result_df_min_temp[(result_df_min_temp['Date'] >= start_date) & (result_df_min_temp['Date'] <= end_date)]


In [13]:

def impute_missing_with_regression(df, target_column, feature_columns):
    # Split data into training and prediction data
    training_data = df.dropna(subset=[target_column] + feature_columns)
    prediction_data = df[df[target_column].isna()]

    # Create and train a regression model
    X_train = training_data[feature_columns]
    y_train = training_data[target_column]
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)

    # Predict missing values
    X_pred = prediction_data[feature_columns]
    predicted_values = regression_model.predict(X_pred)

    # Impute missing values in the original DataFrame
    df.loc[df[target_column].isna(), target_column] = predicted_values




In [15]:
min_temp_dict_by_month1 = {}
unique_months_min_df = filtered_data_df['Date'].dt.month.unique()

for month in unique_months_min_df:
    monthly_min_temp = filtered_data_df[filtered_data_df['Date'].dt.month == month].copy()
    
    monthly_min_temp = monthly_min_temp.reindex()
    
    min_temp_dict_by_month1[month] = monthly_min_temp
    
   

In [17]:
 
 
min_regression_ready_dict = {}
for month_name, month_df in min_temp_dict_by_month1.items():
    # Pivot the DataFrame
    pivoted_df = month_df.pivot_table(index='Station' , columns=['Date'], values='Min T')
    
    # Pivot the DataFrame
    #pivoted_df = month_df.pivot_table(index='Date' , columns=['Station'], values='Min T')
    
    # Replace 'NA' with actual NaN values
    pivoted_df.replace('NaN', np.nan, inplace=True)
    
    # Update the dictionary with the pivoted DataFrame
    min_regression_ready_dict[month_name] = pivoted_df
    



In [19]:

min_regression_1_dict = {}
for month_name, month_df in min_regression_ready_dict.items():
    
    # Step 1: Find the column with the minimum number of missing values
    value_missing_onlyone_column = month_df.isna().sum().idxmin()
    
    #print(value_missing_onlyone_column)
    
    # Step 2: Calculate the average of the selected column
    average_value = month_df[value_missing_onlyone_column].mean()
    
    #print(average_value)

    # Step 3: Replace NaN values in the selected column with the calculated average
    month_df[value_missing_onlyone_column].fillna(average_value, inplace=True)
    min_regression_1_dict[month_name] = month_df


In [21]:
min_regression_12_dict ={}
for month_name, month_df in min_regression_1_dict.items():
    
    # Get a list of all column names except the target columns
    target_columns = month_df.columns[month_df.isna().any()].tolist()
    feature_columns = [col for col in month_df.columns if col not in target_columns]

    # Iterate over target columns and impute missing values using regression
    for target_col in target_columns:
        impute_missing_with_regression(month_df, target_col, feature_columns)
        
    min_regression_12_dict[month_name] = month_df


In [22]:
min_regression_123_dict = {}
for month_name, month_df in min_regression_12_dict.items():
    df_melted_df = month_df.reset_index().melt(id_vars=['Station'], 
                         var_name='Date', value_name='Min T')
    
    df_melted_df = df_melted_df[['Date', 'Station', 'Min T']]
    
    min_regression_123_dict[month_name] = df_melted_df


In [23]:
# Concatenate all DataFrames into a single DataFrame
concatenated_df_filled_1990 = pd.concat(min_regression_123_dict.values(), ignore_index=True)

In [24]:

station_filled_1990 = concatenated_df_filled_1990.set_index('Station')
station_sort_filled_1990 = station_filled_1990.sort_values(by=['Station', 'Date'], ascending=[True, True])


In [26]:
station_sort_filled_1990.to_csv('G:/fresh_start/main_thesis/data/temp_data/min_station_sort_filled_1990.csv')

In [25]:
print(station_sort_filled_1990)

                        Date      Min T
Station                                
1016     1990-01-01 00:00:00  16.865993
1016     1990-01-02 00:00:00  12.336708
1016     1990-01-03 00:00:00  25.611077
1016     1990-01-04 00:00:00   6.503694
1016     1990-01-05 00:00:00  12.335740
...                      ...        ...
1419     2022-12-27 00:00:00   6.000000
1419     2022-12-28 00:00:00   5.500000
1419     2022-12-29 00:00:00   6.500000
1419     2022-12-30 00:00:00   7.500000
1419     2022-12-31 00:00:00   8.500000

[216954 rows x 2 columns]
